# Importing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score

# DATA COLLECTION

In [2]:
##printing dataset
df=pd.read_csv(r"C:\\Users\rushi\Downloads\trainnews.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
#shape
df.shape

(20800, 5)

In [4]:
# checking for null value
null=df.isnull().sum()/len(df)*100
null

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64

In [5]:
## For more accuracy
cat=df.select_dtypes(include="object")
num=df.select_dtypes(include={"int64","float64"})
miss_cat=[var for var in cat if cat[var].isnull().sum()>0]
miss_num=[i for i in num if num[i].isnull().sum()>0]
for i in miss_cat:
    df[i]=cat[i].fillna(cat[i].mode()[0])
for j in miss_num:
    df[j]=cat[j].fillna(cat[j].mean)

In [6]:
#Gives less accuracy
# df=df.fillna("")

In [7]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
df["content"]=df["author"]+" "+df["title"]#+" "+df["text"]
df.content


0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [9]:
x=df.drop(columns="label",axis=1)
y=df["label"]
x.shape,y.shape

((20800, 5), (20800,))

# Stemming

its the proccess where we reduce the word into its root word.it helps us to vectorizing the data (imp to convert big text into shormeans vectorizing)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
portstem=PorterStemmer()

In [12]:
def stemming(x):
    stem=re.sub('[^a-zA-Z]',' ',x)
    stem=stem.lower()
    stem=stem.split()
    stem=[portstem.stem(word) for word in stem if not word in stopwords.words("english")]
    stem='  '.join(stem)
    return stem

In [13]:
df["content"]=df["content"].apply(stemming)

In [14]:
df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us a...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child w...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul ...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci ...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel ex...


In [15]:
x=df["content"].values
y=df["label"].values
x.shape,y.shape

((20800,), (20800,))

In [16]:
x

array(['darrel  lucu  hous  dem  aid  even  see  comey  letter  jason  chaffetz  tweet',
       'daniel  j  flynn  flynn  hillari  clinton  big  woman  campu  breitbart',
       'consortiumnew  com  truth  might  get  fire', ...,
       'michael  j  de  la  merc  rachel  abram  maci  said  receiv  takeov  approach  hudson  bay  new  york  time',
       'alex  ansari  nato  russia  hold  parallel  exercis  balkan',
       'david  swanson  keep  f  aliv'], dtype=object)

In [17]:
y   ##1-Real News and 0-Fake news

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

# Using Tfidf vectorization we convert all data into numeric

In [18]:
vect=TfidfVectorizer()

In [19]:
vect.fit(x)
x=vect.transform(x)

In [20]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.4245253305821674
  (2, 9620)	0.5042960087774311
  (2, 5968)	0.2861705128646944
  (2, 5389)	0.3950996836111487
  (2, 3103)	0.4710451219383643
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420400

# splitting data set to training and test data

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2,stratify=y)

# Train Logistic regression model

In [22]:
model=LogisticRegression()

In [23]:
model.fit(x_train,y_train)

LogisticRegression()

# Evaluate the model

In [24]:
train_x_pred=model.predict(x_train)
acc_train=r2_score(train_x_pred,y_train)
print("acc for train :" ,acc_train)

acc for train : 0.962255296323187


In [25]:
##test data accuracy
test_x_pred=model.predict(x_test)
acc_test=r2_score(test_x_pred,y_test)
print("acc for train :" ,acc_test)

acc for train : 0.9355480817969579


# Prediction of unknown data// Test data

In [26]:
d=x_test[1]
pred_x_test=model.predict(d)
print(pred_x_test)

if pred_x_test==1:
    print("Real news")
else:
    print("Fake")

[0]
Fake


In [27]:
y[1]

0